# Encounter Stitching Demo

This notebook demonstrates how to use the encounter stitching functionality in CLIFpy to link related hospital encounters that occur within a specified time window.

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from clifpy.clif_orchestrator import ClifOrchestrator

In [ ]:
def find_project_root(start=None):
    p = Path(start or Path.cwd())
    for d in [p, *p.parents]:
        if (d / "pyproject.toml").exists() or (d / "clifpy").is_dir():
            return d
    return p

project_root = find_project_root()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
DATA_DIR = (project_root / "clifpy" / "data" / "clif_demo").resolve()
OUTPUT_DIR = (project_root / "examples" / "output").resolve()
FILETYPE = "parquet"
TIMEZONE = "US/Eastern"

print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

## 2. Initialize ClifOrchestrator with Encounter Stitching

The encounter stitching feature can be enabled by setting `stitch_encounter=True` when creating the orchestrator.

In [ ]:
# Initialize orchestrator with encounter stitching enabled
clif = ClifOrchestrator(
    data_directory=str(DATA_DIR),
    filetype=FILETYPE,
    timezone=TIMEZONE,
    output_directory=str(OUTPUT_DIR),
    stitch_encounter=True,  # Enable encounter stitching
    stitch_time_interval=6  # 6-hour window (default)
)

## 3. Load Required Tables

Encounter stitching requires both hospitalization and ADT tables. The stitching will happen automatically during initialization.

In [ ]:
# Load the required tables - stitching happens automatically
clif.initialize(['hospitalization', 'adt'])

## 4. Examine the Results

After stitching, both tables will have a new `encounter_block` column that groups related encounters.

In [ ]:
clif.hospitalization.df

In [ ]:
# Access the encounter mapping
encounter_mapping = clif.get_encounter_mapping()

if encounter_mapping is not None:
    print(f"Total hospitalizations: {len(encounter_mapping)}")
    print(f"Total encounter blocks: {encounter_mapping['encounter_block'].nunique()}")
    print(f"\nEncounter mapping shape: {encounter_mapping.shape}")

## 5. Direct function usage 

In [ ]:
from clifpy import Adt, Hospitalization
from clifpy.utils.stitching_encounters import stitch_encounters

hospitalization = Hospitalization.from_file(
    data_directory=str(DATA_DIR),
    filetype=FILETYPE,
    timezone=TIMEZONE,
    output_directory=str(OUTPUT_DIR),
)
print("Hospitalization data loaded:", hospitalization.df is not None)

adt = Adt.from_file(
    data_directory=str(DATA_DIR),
    filetype=FILETYPE,
    timezone=TIMEZONE,
    output_directory=str(OUTPUT_DIR),
)
print("ADT data loaded:", adt.df is not None)



In [ ]:
hospitalization.df.dtypes

In [ ]:
# Perform stitching
hosp_stitched, adt_stitched, encounter_mapping = stitch_encounters(
    hospitalization=hospitalization.df,
    adt=adt.df,
    time_interval=12  # 12-hour window
)

In [ ]:
hosp_stitched

In [ ]:
encounter_mapping